# Module 3.3: HOG - Histograms of Oriented Gradients

## 學習目標

完成這個 notebook 後，你將能夠：

1. 理解 HOG 特徵的設計原理與物理意義
2. 實作完整的 HOG pipeline（Cell → Block → Normalization）
3. 理解為什麼 HOG 能有效描述物體形狀
4. 準備好在 Module 4 使用 HOG + SVM 進行行人偵測

## 背景知識

HOG (Histograms of Oriented Gradients) 是 Dalal 和 Triggs 在 2005 年 CVPR 提出的特徵描述子，最初用於**行人偵測**。核心想法是：

> 物體的外觀和形狀可以用局部梯度方向的分布來描述

### 為什麼 HOG 有效？

1. **邊緣方向**：梯度方向捕捉了邊緣的走向，而邊緣定義了物體的形狀
2. **統計特性**：用直方圖統計，對小位移不敏感
3. **局部正規化**：Block normalization 處理光照變化

### HOG vs SIFT

| 特性 | HOG | SIFT |
|------|-----|------|
| 用途 | Dense features for detection | Sparse keypoint matching |
| 計算位置 | 整張圖（滑動窗口） | 只在關鍵點 |
| 尺度不變 | 否（用固定尺度） | 是（多尺度） |
| 旋轉不變 | 否（用0°-180°） | 是（主方向對齊） |

---

## 參考論文

- Dalal & Triggs, *"Histograms of Oriented Gradients for Human Detection"*, CVPR 2005

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# 設定 matplotlib
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.cmap'] = 'gray'

print("環境設定完成！")
print(f"NumPy 版本: {np.__version__}")

---

## Part 1: HOG 架構概覽

### HOG 的處理流程

```
輸入圖像 (64x128 pixels for pedestrian detection)
    ↓
(Optional) Gamma Correction
    ↓
Compute Gradients (Gx, Gy)
    ↓
Divide into Cells (8x8 pixels each)
    ↓
Compute Cell Histograms (9 bins, 0°-180°)
    ↓
Group Cells into Blocks (2x2 cells)
    ↓
Block Normalization (L2-norm)
    ↓
Concatenate into Feature Vector
```

### 關鍵參數（Dalal & Triggs 原始設定）

| 參數 | 值 | 說明 |
|------|-----|------|
| Cell size | 8×8 pixels | 局部梯度統計單位 |
| Block size | 2×2 cells (16×16 pixels) | 正規化範圍 |
| Block stride | 1 cell (8 pixels) | 相鄰 block 重疊 |
| Num bins | 9 | 方向量化（0°-180°，每 bin 20°）|
| Window size | 64×128 pixels | 行人偵測窗口 |

In [ ]:
# 建立測試圖像
def create_test_images():
    """
    建立用於測試 HOG 的圖像
    """
    # 簡單的人形剪影
    def create_person_silhouette(height=128, width=64):
        img = np.ones((height, width), dtype=np.float64) * 255
        
        # Head (circle)
        cy, cx = 15, 32
        radius = 8
        for y in range(height):
            for x in range(width):
                if (y - cy)**2 + (x - cx)**2 < radius**2:
                    img[y, x] = 50
        
        # Torso
        img[25:70, 22:42] = 50
        
        # Left arm
        for i in range(20):
            y = 30 + i
            x = 22 - i // 2
            if 0 <= x < width and 0 <= y < height:
                img[y:y+4, max(0, x-2):x+2] = 50
        
        # Right arm
        for i in range(20):
            y = 30 + i
            x = 42 + i // 2
            if 0 <= x < width and 0 <= y < height:
                img[y:y+4, x:min(width, x+4)] = 50
        
        # Left leg
        img[70:120, 22:30] = 50
        
        # Right leg  
        img[70:120, 34:42] = 50
        
        return img
    
    # 垂直邊緣
    def create_vertical_edges(height=64, width=64):
        img = np.zeros((height, width), dtype=np.float64)
        for i in range(0, width, 16):
            img[:, i:i+8] = 255
        return img
    
    # 水平邊緣
    def create_horizontal_edges(height=64, width=64):
        img = np.zeros((height, width), dtype=np.float64)
        for i in range(0, height, 16):
            img[i:i+8, :] = 255
        return img
    
    # 對角線
    def create_diagonal(height=64, width=64):
        img = np.zeros((height, width), dtype=np.float64)
        for y in range(height):
            for x in range(width):
                if (x + y) % 32 < 16:
                    img[y, x] = 255
        return img
    
    return {
        'person': create_person_silhouette(),
        'vertical': create_vertical_edges(),
        'horizontal': create_horizontal_edges(),
        'diagonal': create_diagonal()
    }

test_images = create_test_images()

# 顯示測試圖像
fig, axes = plt.subplots(1, 4, figsize=(14, 4))
for ax, (name, img) in zip(axes, test_images.items()):
    ax.imshow(img, cmap='gray')
    ax.set_title(f'{name}\n{img.shape}')
    ax.axis('off')
plt.suptitle('測試圖像', fontsize=14)
plt.tight_layout()
plt.show()

---

## Part 2: 梯度計算

### 梯度計算方法

HOG 使用簡單的 1D centered difference：

$$G_x(x, y) = I(x+1, y) - I(x-1, y)$$
$$G_y(x, y) = I(x, y+1) - I(x, y-1)$$

這比 Sobel 簡單，但在行人偵測任務上效果足夠。

### 梯度大小與方向

$$\text{magnitude} = \sqrt{G_x^2 + G_y^2}$$

$$\text{direction} = \arctan\left(\frac{G_y}{G_x}\right)$$

**注意**：HOG 使用 **unsigned gradient**（0°-180°），因為邊緣的方向不區分正負。

In [ ]:
def compute_gradients(image):
    """
    計算圖像的梯度大小和方向
    
    使用 centered difference: Gx = I(x+1) - I(x-1), Gy = I(y+1) - I(y-1)
    
    Parameters
    ----------
    image : np.ndarray
        灰階圖像，shape (H, W)
    
    Returns
    -------
    magnitude : np.ndarray
        梯度大小，shape (H, W)
    direction : np.ndarray
        梯度方向（度），範圍 [0, 180)，shape (H, W)
    """
    H, W = image.shape
    
    # 計算 Gx（水平梯度）
    # Gx[y, x] = I[y, x+1] - I[y, x-1]
    gx = np.zeros_like(image, dtype=np.float64)
    gx[:, 1:-1] = image[:, 2:] - image[:, :-2]
    # 邊界處理：用單側差分
    gx[:, 0] = image[:, 1] - image[:, 0]
    gx[:, -1] = image[:, -1] - image[:, -2]
    
    # 計算 Gy（垂直梯度）
    # Gy[y, x] = I[y+1, x] - I[y-1, x]
    gy = np.zeros_like(image, dtype=np.float64)
    gy[1:-1, :] = image[2:, :] - image[:-2, :]
    # 邊界處理
    gy[0, :] = image[1, :] - image[0, :]
    gy[-1, :] = image[-1, :] - image[-2, :]
    
    # 梯度大小
    magnitude = np.sqrt(gx**2 + gy**2)
    
    # 梯度方向（弧度）
    direction_rad = np.arctan2(gy, gx)
    
    # 轉換為度數 [0, 180)
    # arctan2 返回 [-π, π]，我們需要 [0, 180)
    direction_deg = np.degrees(direction_rad)
    
    # 將負角度轉換為正（利用 180° 對稱性）
    # 例如：-30° 等價於 150°，但因為是 unsigned，我們把 [-180, 0) 映射到 [0, 180)
    direction_deg = direction_deg % 180
    
    return magnitude, direction_deg

# 測試梯度計算
print("=== 測試梯度計算 ===")

# 對不同測試圖像計算梯度
fig, axes = plt.subplots(3, 4, figsize=(14, 10))

for col, (name, img) in enumerate(test_images.items()):
    mag, direction = compute_gradients(img)
    
    axes[0, col].imshow(img, cmap='gray')
    axes[0, col].set_title(f'{name}\nOriginal')
    axes[0, col].axis('off')
    
    axes[1, col].imshow(mag, cmap='hot')
    axes[1, col].set_title('Magnitude')
    axes[1, col].axis('off')
    
    # 用 HSV 顯示方向（色相=方向）
    im = axes[2, col].imshow(direction, cmap='hsv', vmin=0, vmax=180)
    axes[2, col].set_title('Direction (0°-180°)')
    axes[2, col].axis('off')

plt.suptitle('梯度計算結果', fontsize=14)
plt.tight_layout()
plt.show()

# 驗證：垂直邊緣的方向應該接近 0° 或 180°
# 水平邊緣的方向應該接近 90°
print("\n驗證方向計算：")
mag_v, dir_v = compute_gradients(test_images['vertical'])
mag_h, dir_h = compute_gradients(test_images['horizontal'])

# 只看有梯度的位置
v_dirs = dir_v[mag_v > 10]
h_dirs = dir_h[mag_h > 10]

print(f"垂直邊緣的主要方向: {np.median(v_dirs):.1f}° (期望: 0° 或接近 180°)")
print(f"水平邊緣的主要方向: {np.median(h_dirs):.1f}° (期望: 90°)")

---

## Part 3: Cell Histogram

### Cell 的概念

把圖像分成小區域（cell），每個 cell 計算一個梯度方向直方圖。

- **Cell 大小**：8×8 pixels（原論文設定）
- **Bins 數量**：9 bins，每個 bin 涵蓋 20°（0°-180° / 9 = 20°）

### 直方圖計算（Voting）

每個像素的梯度「投票」到對應的 bin，權重是梯度大小：

$$\text{hist}[\text{bin}] += \text{magnitude}$$

其中 $\text{bin} = \lfloor \text{direction} / 20 \rfloor$

### Soft Binning（雙線性內插）

原論文使用 soft binning：如果方向落在兩個 bin 之間，按比例分配到兩個 bin。

例如：方向 = 35°
- bin 1（20°-40°）得到 75% 的權重
- bin 2（40°-60°）得到 25% 的權重

In [ ]:
def compute_cell_histogram_hard(magnitude, direction, num_bins=9):
    """
    計算 cell 的梯度方向直方圖（Hard binning）
    
    每個像素的梯度投票到最近的 bin
    
    Parameters
    ----------
    magnitude : np.ndarray
        梯度大小，shape (cell_size, cell_size)
    direction : np.ndarray
        梯度方向（度），範圍 [0, 180)，shape (cell_size, cell_size)
    num_bins : int
        直方圖的 bin 數量（預設 9）
    
    Returns
    -------
    hist : np.ndarray
        直方圖，shape (num_bins,)
    """
    # 每個 bin 的寬度（度）
    bin_width = 180.0 / num_bins  # 20° for 9 bins
    
    # 計算每個像素屬於哪個 bin
    # direction 範圍是 [0, 180)，bin 編號是 [0, num_bins-1]
    bin_indices = (direction / bin_width).astype(int)
    
    # 處理邊界情況：direction = 180 應該進入 bin 0（循環）
    bin_indices = bin_indices % num_bins
    
    # 累加直方圖
    hist = np.zeros(num_bins, dtype=np.float64)
    for i in range(num_bins):
        mask = (bin_indices == i)
        hist[i] = np.sum(magnitude[mask])
    
    return hist


def compute_cell_histogram_soft(magnitude, direction, num_bins=9):
    """
    計算 cell 的梯度方向直方圖（Soft binning / 雙線性內插）
    
    每個像素的梯度按比例投票到相鄰的兩個 bins
    
    Parameters
    ----------
    magnitude : np.ndarray
        梯度大小，shape (cell_size, cell_size)
    direction : np.ndarray
        梯度方向（度），範圍 [0, 180)，shape (cell_size, cell_size)
    num_bins : int
        直方圖的 bin 數量（預設 9）
    
    Returns
    -------
    hist : np.ndarray
        直方圖，shape (num_bins,)
    """
    bin_width = 180.0 / num_bins  # 20° for 9 bins
    
    # 計算連續的 bin 位置
    # 例如 direction = 35° → bin_pos = 1.75
    bin_pos = direction / bin_width
    
    # 左邊的 bin 和右邊的 bin
    bin_left = np.floor(bin_pos).astype(int)
    bin_right = bin_left + 1
    
    # 內插權重
    # weight_right = 離左邊 bin 中心的比例
    weight_right = bin_pos - bin_left  # 0.75 for the example
    weight_left = 1 - weight_right      # 0.25
    
    # 循環處理（bin num_bins 等於 bin 0）
    bin_left = bin_left % num_bins
    bin_right = bin_right % num_bins
    
    # 累加直方圖
    hist = np.zeros(num_bins, dtype=np.float64)
    
    # 展平處理
    mag_flat = magnitude.flatten()
    bl_flat = bin_left.flatten()
    br_flat = bin_right.flatten()
    wl_flat = weight_left.flatten()
    wr_flat = weight_right.flatten()
    
    for i in range(len(mag_flat)):
        hist[bl_flat[i]] += mag_flat[i] * wl_flat[i]
        hist[br_flat[i]] += mag_flat[i] * wr_flat[i]
    
    return hist


# 測試 cell histogram
print("=== 測試 Cell Histogram ===")

# 建立簡單的測試案例
cell_size = 8

# 測試 1：單一方向（水平邊緣，方向 90°）
mag_test1 = np.ones((cell_size, cell_size))
dir_test1 = np.full((cell_size, cell_size), 90.0)

hist_hard = compute_cell_histogram_hard(mag_test1, dir_test1)
hist_soft = compute_cell_histogram_soft(mag_test1, dir_test1)

print("\n測試 1：全部方向 = 90°")
print(f"Hard binning: {hist_hard}")
print(f"Soft binning: {hist_soft}")
print(f"期望：bin 4 (80°-100°) 有最大值")

# 測試 2：方向 = 35°（在 bin 1 和 bin 2 之間）
dir_test2 = np.full((cell_size, cell_size), 35.0)

hist_hard2 = compute_cell_histogram_hard(mag_test1, dir_test2)
hist_soft2 = compute_cell_histogram_soft(mag_test1, dir_test2)

print("\n測試 2：全部方向 = 35°（在 20°-40° bin 內）")
print(f"Hard binning: {hist_hard2}")
print(f"Soft binning: {hist_soft2}")
print(f"Hard binning 只有 bin 1，Soft binning 分配到 bin 1 和 bin 2")

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

bin_centers = np.arange(9) * 20 + 10  # 10°, 30°, 50°, ...

axes[0].bar(bin_centers, hist_hard2, width=15, alpha=0.7, label='Hard')
axes[0].bar(bin_centers, hist_soft2, width=10, alpha=0.7, label='Soft')
axes[0].set_xlabel('方向（度）')
axes[0].set_ylabel('累積權重')
axes[0].set_title('方向 = 35° 的直方圖比較')
axes[0].legend()
axes[0].set_xticks(bin_centers)

# 測試 3：用真實圖像的一個 cell
person_img = test_images['person']
mag, direction = compute_gradients(person_img)

# 取頭部區域的一個 cell
cell_mag = mag[8:16, 24:32]
cell_dir = direction[8:16, 24:32]

hist_cell = compute_cell_histogram_soft(cell_mag, cell_dir)

axes[1].bar(bin_centers, hist_cell, width=15)
axes[1].set_xlabel('方向（度）')
axes[1].set_ylabel('累積權重')
axes[1].set_title('人形剪影頭部區域的方向直方圖')
axes[1].set_xticks(bin_centers)

plt.tight_layout()
plt.show()

---

## Part 4: Block Normalization

### 為什麼需要 Block Normalization？

1. **光照變化**：不同光照下，梯度大小會不同，但形狀（方向分布）不變
2. **對比度變化**：高對比度圖像的梯度較大

正規化讓特徵對這些變化具有不變性。

### Block 結構

- 1 block = 2×2 cells = 16×16 pixels
- Block 之間有重疊（stride = 1 cell）
- 每個 block 內的所有 cell histograms 串接後一起正規化

```
Block 內容：
[ cell(0,0) | cell(0,1) ]
[ cell(1,0) | cell(1,1) ]

串接後：[hist(0,0), hist(0,1), hist(1,0), hist(1,1)]
維度：9 × 4 = 36
```

### L2 Normalization

$$v_{\text{normalized}} = \frac{v}{\sqrt{\|v\|_2^2 + \epsilon}}$$

其中 $\epsilon$ 是小常數（避免除以零）。

In [ ]:
def normalize_block_l2(block_features, eps=1e-5):
    """
    L2 正規化 block 特徵
    
    Parameters
    ----------
    block_features : np.ndarray
        Block 的特徵向量（串接的 cell histograms）
    eps : float
        避免除以零的小常數
    
    Returns
    -------
    np.ndarray
        正規化後的特徵向量
    """
    norm = np.sqrt(np.sum(block_features ** 2) + eps)
    return block_features / norm


def normalize_block_l2_hys(block_features, eps=1e-5, clip=0.2):
    """
    L2-Hys 正規化（L2 + clipping + renormalization）
    
    這是 Dalal & Triggs 推薦的方法，效果最好
    
    Parameters
    ----------
    block_features : np.ndarray
        Block 的特徵向量
    eps : float
        避免除以零的小常數
    clip : float
        Clipping 閾值
    
    Returns
    -------
    np.ndarray
        正規化後的特徵向量
    """
    # 第一次 L2 正規化
    norm = np.sqrt(np.sum(block_features ** 2) + eps)
    normalized = block_features / norm
    
    # Clipping：限制最大值
    clipped = np.minimum(normalized, clip)
    
    # 第二次 L2 正規化
    norm2 = np.sqrt(np.sum(clipped ** 2) + eps)
    return clipped / norm2


# 測試正規化
print("=== 測試 Block Normalization ===")

# 模擬一個 block 的特徵（2×2 cells，每個 9 bins）
block_raw = np.array([
    100, 200, 50, 10, 5, 2, 1, 0, 0,   # cell 1
    80, 150, 40, 8, 4, 2, 1, 0, 0,     # cell 2  
    90, 180, 45, 9, 4, 2, 1, 0, 0,     # cell 3
    85, 170, 42, 8, 4, 2, 1, 0, 0      # cell 4
], dtype=np.float64)

print(f"原始 block 特徵範圍: [{block_raw.min():.1f}, {block_raw.max():.1f}]")
print(f"原始 L2 norm: {np.linalg.norm(block_raw):.1f}")

normalized_l2 = normalize_block_l2(block_raw)
normalized_l2_hys = normalize_block_l2_hys(block_raw)

print(f"\nL2 正規化後範圍: [{normalized_l2.min():.4f}, {normalized_l2.max():.4f}]")
print(f"L2 正規化後 norm: {np.linalg.norm(normalized_l2):.4f}")

print(f"\nL2-Hys 正規化後範圍: [{normalized_l2_hys.min():.4f}, {normalized_l2_hys.max():.4f}]")
print(f"L2-Hys 正規化後 norm: {np.linalg.norm(normalized_l2_hys):.4f}")

# 視覺化
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

axes[0].bar(range(len(block_raw)), block_raw)
axes[0].set_title('原始 Block 特徵')
axes[0].set_xlabel('特徵維度')

axes[1].bar(range(len(normalized_l2)), normalized_l2)
axes[1].set_title('L2 正規化後')
axes[1].set_xlabel('特徵維度')

axes[2].bar(range(len(normalized_l2_hys)), normalized_l2_hys)
axes[2].set_title('L2-Hys 正規化後')
axes[2].set_xlabel('特徵維度')

plt.tight_layout()
plt.show()

print("\n觀察：L2-Hys 的 clipping 降低了大值的影響，讓特徵分布更均勻")

---

## Part 5: 完整 HOG Pipeline

現在我們把所有部分組合起來，實作完整的 HOG 特徵擷取。

### 特徵向量維度計算

假設：
- 圖像大小：64×128 pixels
- Cell size：8×8 pixels → 8×16 cells
- Block size：2×2 cells
- Block stride：1 cell
- Blocks：(8-1)×(16-1) = 7×15 = 105 blocks
- 每個 block：2×2×9 = 36 維
- 總維度：105×36 = **3780 維**

這是標準行人偵測 HOG 的維度。

In [ ]:
def compute_hog(image, cell_size=8, block_size=2, num_bins=9, 
                use_soft_binning=True, normalization='L2-Hys'):
    """
    計算完整的 HOG 特徵
    
    Parameters
    ----------
    image : np.ndarray
        灰階圖像，shape (H, W)，H 和 W 需為 cell_size 的倍數
    cell_size : int
        Cell 大小（pixels），預設 8
    block_size : int
        Block 大小（cells），預設 2（即 2×2 cells）
    num_bins : int
        方向直方圖的 bin 數量，預設 9
    use_soft_binning : bool
        是否使用 soft binning（雙線性內插）
    normalization : str
        正規化方法：'L2', 'L2-Hys', 或 'none'
    
    Returns
    -------
    features : np.ndarray
        HOG 特徵向量
    cell_hists : np.ndarray
        所有 cell 的直方圖，shape (num_cells_y, num_cells_x, num_bins)
    """
    H, W = image.shape
    
    # 確認尺寸是 cell_size 的倍數
    assert H % cell_size == 0, f"H={H} must be multiple of cell_size={cell_size}"
    assert W % cell_size == 0, f"W={W} must be multiple of cell_size={cell_size}"
    
    # Cell 數量
    num_cells_y = H // cell_size
    num_cells_x = W // cell_size
    
    # Step 1: 計算梯度
    magnitude, direction = compute_gradients(image.astype(np.float64))
    
    # Step 2: 計算所有 cell 的直方圖
    cell_hists = np.zeros((num_cells_y, num_cells_x, num_bins))
    
    hist_func = compute_cell_histogram_soft if use_soft_binning else compute_cell_histogram_hard
    
    for cy in range(num_cells_y):
        for cx in range(num_cells_x):
            # Cell 的像素範圍
            y_start = cy * cell_size
            y_end = y_start + cell_size
            x_start = cx * cell_size
            x_end = x_start + cell_size
            
            # 取出 cell 的梯度
            cell_mag = magnitude[y_start:y_end, x_start:x_end]
            cell_dir = direction[y_start:y_end, x_start:x_end]
            
            # 計算直方圖
            cell_hists[cy, cx] = hist_func(cell_mag, cell_dir, num_bins)
    
    # Step 3: Block normalization
    # Block 數量
    num_blocks_y = num_cells_y - block_size + 1
    num_blocks_x = num_cells_x - block_size + 1
    
    # 特徵維度
    block_dim = block_size * block_size * num_bins  # 36 for 2×2 blocks, 9 bins
    
    features = []
    
    for by in range(num_blocks_y):
        for bx in range(num_blocks_x):
            # 取出 block 內的所有 cell histograms
            block_hists = cell_hists[by:by+block_size, bx:bx+block_size]
            
            # 串接成一個向量
            block_vec = block_hists.flatten()
            
            # 正規化
            if normalization == 'L2':
                block_vec = normalize_block_l2(block_vec)
            elif normalization == 'L2-Hys':
                block_vec = normalize_block_l2_hys(block_vec)
            # else: 不正規化
            
            features.append(block_vec)
    
    # 串接所有 block 的特徵
    features = np.concatenate(features)
    
    return features, cell_hists


# 測試完整 HOG
print("=== 測試完整 HOG Pipeline ===")

# 使用人形剪影圖像
person_img = test_images['person']  # 128×64

# 計算 HOG
features, cell_hists = compute_hog(person_img)

print(f"\n圖像大小: {person_img.shape}")
print(f"Cell 數量: {cell_hists.shape[0]}×{cell_hists.shape[1]} = {cell_hists.shape[0] * cell_hists.shape[1]}")
print(f"HOG 特徵維度: {features.shape[0]}")

# 驗證維度計算
num_cells_y, num_cells_x = cell_hists.shape[:2]
num_blocks = (num_cells_y - 1) * (num_cells_x - 1)
expected_dim = num_blocks * 36
print(f"預期維度: {num_blocks} blocks × 36 = {expected_dim}")

print(f"\n特徵值範圍: [{features.min():.4f}, {features.max():.4f}]")
print(f"特徵 L2 norm: {np.linalg.norm(features):.2f}")

In [ ]:
# HOG 視覺化

def visualize_hog(image, cell_hists, cell_size=8, scale=2.0):
    """
    視覺化 HOG 特徵
    
    在每個 cell 中畫出方向直方圖（用線段表示）
    
    Parameters
    ----------
    image : np.ndarray
        原始圖像
    cell_hists : np.ndarray
        Cell 直方圖，shape (num_cells_y, num_cells_x, num_bins)
    cell_size : int
        Cell 大小
    scale : float
        線段長度縮放因子
    """
    num_cells_y, num_cells_x, num_bins = cell_hists.shape
    
    # 計算每個 bin 的方向角度
    bin_width = 180.0 / num_bins
    angles = np.arange(num_bins) * bin_width + bin_width / 2  # bin 中心
    angles_rad = np.radians(angles)
    
    # 建立視覺化圖像
    fig, ax = plt.subplots(figsize=(10, 12))
    ax.imshow(image, cmap='gray')
    
    # 正規化直方圖用於顯示
    max_val = cell_hists.max() + 1e-5
    
    for cy in range(num_cells_y):
        for cx in range(num_cells_x):
            # Cell 中心
            center_y = cy * cell_size + cell_size / 2
            center_x = cx * cell_size + cell_size / 2
            
            hist = cell_hists[cy, cx]
            
            # 畫每個 bin 的線段
            for b in range(num_bins):
                if hist[b] < max_val * 0.05:  # 跳過太小的值
                    continue
                
                # 線段長度與直方圖值成比例
                length = (hist[b] / max_val) * cell_size * scale / 2
                
                # 計算線段端點
                angle = angles_rad[b]
                dx = length * np.cos(angle)
                dy = length * np.sin(angle)
                
                # 畫線段（雙向）
                ax.plot([center_x - dx, center_x + dx], 
                       [center_y - dy, center_y + dy],
                       'g-', linewidth=1, alpha=0.7)
    
    ax.set_title('HOG 視覺化（綠線表示梯度方向強度）')
    ax.axis('off')
    plt.tight_layout()
    return fig, ax


# 視覺化不同測試圖像的 HOG
fig, axes = plt.subplots(2, 4, figsize=(16, 10))

for col, (name, img) in enumerate(test_images.items()):
    # 調整大小為 cell_size 的倍數
    if name == 'person':
        test_img = img  # 已經是 128×64
    else:
        test_img = img  # 64×64
    
    # 顯示原始圖像
    axes[0, col].imshow(test_img, cmap='gray')
    axes[0, col].set_title(f'{name}\nOriginal')
    axes[0, col].axis('off')
    
    # 計算 HOG
    features, cell_hists = compute_hog(test_img)
    
    # 顯示 cell histogram 熱圖（主要方向）
    # 找每個 cell 的主要方向
    dominant_bins = np.argmax(cell_hists, axis=2)
    dominant_angles = dominant_bins * 20 + 10  # bin 中心角度
    
    im = axes[1, col].imshow(dominant_angles, cmap='hsv', vmin=0, vmax=180)
    axes[1, col].set_title(f'Dominant Direction\nFeature dim: {len(features)}')
    axes[1, col].axis('off')

plt.suptitle('不同圖像的 HOG 特徵', fontsize=14)
plt.tight_layout()
plt.show()

# 詳細視覺化人形圖像
print("\n詳細視覺化人形剪影的 HOG：")
visualize_hog(test_images['person'], cell_hists)
plt.show()

---

## Part 6: HOG 對光照變化的不變性

讓我們驗證 block normalization 如何幫助處理光照變化。

In [ ]:
# 測試光照不變性
print("=== 測試 HOG 光照不變性 ===")

person_img = test_images['person']

# 建立不同光照版本
bright_img = np.clip(person_img * 1.5, 0, 255)
dark_img = person_img * 0.5
contrast_img = np.clip((person_img - 128) * 1.5 + 128, 0, 255)

# 計算各版本的 HOG
features_normal, _ = compute_hog(person_img, normalization='L2-Hys')
features_bright, _ = compute_hog(bright_img, normalization='L2-Hys')
features_dark, _ = compute_hog(dark_img, normalization='L2-Hys')
features_contrast, _ = compute_hog(contrast_img, normalization='L2-Hys')

# 也計算不正規化的版本
features_normal_raw, _ = compute_hog(person_img, normalization='none')
features_bright_raw, _ = compute_hog(bright_img, normalization='none')
features_dark_raw, _ = compute_hog(dark_img, normalization='none')

# 計算相似度（Cosine Similarity）
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

print("\n=== 有 L2-Hys 正規化 ===")
print(f"Normal vs Bright: {cosine_similarity(features_normal, features_bright):.4f}")
print(f"Normal vs Dark:   {cosine_similarity(features_normal, features_dark):.4f}")
print(f"Normal vs High Contrast: {cosine_similarity(features_normal, features_contrast):.4f}")

print("\n=== 無正規化 ===")
print(f"Normal vs Bright: {cosine_similarity(features_normal_raw, features_bright_raw):.4f}")
print(f"Normal vs Dark:   {cosine_similarity(features_normal_raw, features_dark_raw):.4f}")

print("\n結論：有正規化時，不同光照條件的特徵幾乎相同（接近 1.0）")

# 視覺化
fig, axes = plt.subplots(2, 4, figsize=(14, 8))

images = [person_img, bright_img, dark_img, contrast_img]
titles = ['Normal', 'Bright (×1.5)', 'Dark (×0.5)', 'High Contrast']

for i, (img, title) in enumerate(zip(images, titles)):
    axes[0, i].imshow(img, cmap='gray', vmin=0, vmax=255)
    axes[0, i].set_title(title)
    axes[0, i].axis('off')

# 顯示特徵差異
features_list = [features_normal, features_bright, features_dark, features_contrast]

for i, (feat, title) in enumerate(zip(features_list, titles)):
    diff = feat - features_normal
    axes[1, i].bar(range(0, len(diff), 10), diff[::10])  # 只顯示部分
    axes[1, i].set_title(f'特徵差異 (vs Normal)\nMax diff: {np.abs(diff).max():.4f}')
    axes[1, i].set_ylim(-0.1, 0.1)

plt.suptitle('HOG 光照不變性測試', fontsize=14)
plt.tight_layout()
plt.show()

---

## Part 7: 向量化實作

上面的實作用了很多 for 迴圈，對大圖像會很慢。這裡提供一個更高效的向量化版本。

In [ ]:
def compute_hog_vectorized(image, cell_size=8, block_size=2, num_bins=9, eps=1e-5):
    """
    向量化的 HOG 實作
    
    Parameters
    ----------
    image : np.ndarray
        灰階圖像，shape (H, W)
    cell_size : int
        Cell 大小（pixels）
    block_size : int  
        Block 大小（cells）
    num_bins : int
        方向直方圖的 bin 數量
    eps : float
        數值穩定性常數
    
    Returns
    -------
    features : np.ndarray
        HOG 特徵向量
    """
    H, W = image.shape
    img = image.astype(np.float64)
    
    # Step 1: 計算梯度（向量化）
    # Gx: 水平梯度
    gx = np.zeros((H, W), dtype=np.float64)
    gx[:, 1:-1] = img[:, 2:] - img[:, :-2]
    gx[:, 0] = img[:, 1] - img[:, 0]
    gx[:, -1] = img[:, -1] - img[:, -2]
    
    # Gy: 垂直梯度
    gy = np.zeros((H, W), dtype=np.float64)
    gy[1:-1, :] = img[2:, :] - img[:-2, :]
    gy[0, :] = img[1, :] - img[0, :]
    gy[-1, :] = img[-1, :] - img[-2, :]
    
    # 梯度大小和方向
    magnitude = np.sqrt(gx**2 + gy**2)
    direction = np.degrees(np.arctan2(gy, gx)) % 180  # [0, 180)
    
    # Step 2: 計算 cell histograms（向量化 soft binning）
    num_cells_y = H // cell_size
    num_cells_x = W // cell_size
    bin_width = 180.0 / num_bins
    
    # 計算 bin 位置和權重
    bin_pos = direction / bin_width
    bin_left = np.floor(bin_pos).astype(int) % num_bins
    bin_right = (bin_left + 1) % num_bins
    weight_right = bin_pos - np.floor(bin_pos)
    weight_left = 1 - weight_right
    
    # 預計算 cell histograms
    cell_hists = np.zeros((num_cells_y, num_cells_x, num_bins))
    
    # 重塑為 cells
    magnitude_cells = magnitude.reshape(num_cells_y, cell_size, num_cells_x, cell_size)
    magnitude_cells = magnitude_cells.transpose(0, 2, 1, 3)  # (cy, cx, h, w)
    
    bin_left_cells = bin_left.reshape(num_cells_y, cell_size, num_cells_x, cell_size)
    bin_left_cells = bin_left_cells.transpose(0, 2, 1, 3)
    
    bin_right_cells = bin_right.reshape(num_cells_y, cell_size, num_cells_x, cell_size)
    bin_right_cells = bin_right_cells.transpose(0, 2, 1, 3)
    
    weight_left_cells = weight_left.reshape(num_cells_y, cell_size, num_cells_x, cell_size)
    weight_left_cells = weight_left_cells.transpose(0, 2, 1, 3)
    
    weight_right_cells = weight_right.reshape(num_cells_y, cell_size, num_cells_x, cell_size)
    weight_right_cells = weight_right_cells.transpose(0, 2, 1, 3)
    
    # 累加直方圖（用 np.add.at 向量化）
    for cy in range(num_cells_y):
        for cx in range(num_cells_x):
            mag = magnitude_cells[cy, cx].flatten()
            bl = bin_left_cells[cy, cx].flatten()
            br = bin_right_cells[cy, cx].flatten()
            wl = weight_left_cells[cy, cx].flatten()
            wr = weight_right_cells[cy, cx].flatten()
            
            np.add.at(cell_hists[cy, cx], bl, mag * wl)
            np.add.at(cell_hists[cy, cx], br, mag * wr)
    
    # Step 3: Block normalization（L2-Hys）
    num_blocks_y = num_cells_y - block_size + 1
    num_blocks_x = num_cells_x - block_size + 1
    block_dim = block_size * block_size * num_bins
    
    features = np.zeros((num_blocks_y, num_blocks_x, block_dim))
    
    for by in range(num_blocks_y):
        for bx in range(num_blocks_x):
            block = cell_hists[by:by+block_size, bx:bx+block_size].flatten()
            
            # L2-Hys normalization
            norm = np.sqrt(np.sum(block**2) + eps)
            block = block / norm
            block = np.minimum(block, 0.2)  # Clip
            norm2 = np.sqrt(np.sum(block**2) + eps)
            block = block / norm2
            
            features[by, bx] = block
    
    return features.flatten()


# 比較兩種實作
print("=== 比較原始實作與向量化實作 ===")

import time

person_img = test_images['person']

# 原始實作
start = time.time()
features_orig, _ = compute_hog(person_img)
time_orig = time.time() - start

# 向量化實作
start = time.time()
features_vec = compute_hog_vectorized(person_img)
time_vec = time.time() - start

print(f"原始實作時間: {time_orig*1000:.2f} ms")
print(f"向量化實作時間: {time_vec*1000:.2f} ms")
print(f"加速比: {time_orig/time_vec:.2f}x")

# 驗證結果一致性
diff = np.abs(features_orig - features_vec)
print(f"\n最大差異: {diff.max():.10f}")
print(f"平均差異: {diff.mean():.10f}")
print(f"結果是否一致: {np.allclose(features_orig, features_vec, atol=1e-8)}")

---

## 練習題

### 練習 1：理解 HOG 參數的影響

**任務**：分析不同參數對 HOG 特徵的影響

1. Cell size：4×4 vs 8×8 vs 16×16
2. Num bins：6 vs 9 vs 18
3. Block normalization：L2 vs L2-Hys vs None

**提示**：
- 較小的 cell 捕捉更細節的資訊，但特徵維度更高
- 更多的 bins 提供更精確的方向資訊，但需要更多資料支持
- 正規化對光照變化很重要

In [ ]:
# 練習 1 解答

def analyze_hog_parameters(image):
    """
    分析不同 HOG 參數的影響
    """
    results = {}
    
    # 1. Cell size 比較（確保圖像大小相容）
    print("=== Cell Size 比較 ===")
    for cell_size in [4, 8, 16]:
        # 調整圖像大小為 cell_size 的倍數
        H, W = image.shape
        new_H = (H // cell_size) * cell_size
        new_W = (W // cell_size) * cell_size
        img_resized = image[:new_H, :new_W]
        
        features, cell_hists = compute_hog(img_resized, cell_size=cell_size)
        results[f'cell_{cell_size}'] = features
        
        print(f"Cell size {cell_size}×{cell_size}:")
        print(f"  圖像大小: {img_resized.shape}")
        print(f"  Cell 數量: {cell_hists.shape[0]}×{cell_hists.shape[1]}")
        print(f"  特徵維度: {len(features)}")
        print()
    
    # 2. Num bins 比較
    print("\n=== Num Bins 比較 ===")
    for num_bins in [6, 9, 18]:
        features, _ = compute_hog(image, num_bins=num_bins)
        results[f'bins_{num_bins}'] = features
        
        print(f"Num bins {num_bins}:")
        print(f"  每個 bin 的角度範圍: {180/num_bins:.1f}°")
        print(f"  特徵維度: {len(features)}")
        print()
    
    # 3. Normalization 比較
    print("\n=== Normalization 比較 ===")
    for norm in ['none', 'L2', 'L2-Hys']:
        features, _ = compute_hog(image, normalization=norm)
        results[f'norm_{norm}'] = features
        
        print(f"Normalization: {norm}")
        print(f"  特徵值範圍: [{features.min():.4f}, {features.max():.4f}]")
        print(f"  特徵 L2 norm: {np.linalg.norm(features):.2f}")
        print()
    
    return results

# 執行分析
results = analyze_hog_parameters(test_images['person'])

# 視覺化不同正規化的效果
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for ax, norm in zip(axes, ['none', 'L2', 'L2-Hys']):
    features = results[f'norm_{norm}']
    ax.hist(features, bins=50, alpha=0.7)
    ax.set_title(f'Normalization: {norm}')
    ax.set_xlabel('Feature value')
    ax.set_ylabel('Count')

plt.suptitle('不同正規化方法的特徵值分布', fontsize=14)
plt.tight_layout()
plt.show()

### 練習 2：實作 Gamma Correction

**任務**：在 HOG pipeline 前加入 Gamma correction 預處理

Gamma correction 公式：
$$I_{\text{corrected}} = I^\gamma$$

其中 $\gamma$ 通常取 0.5（相當於開根號）。

**提示**：
- Gamma correction 可以壓縮高亮度值，增強低亮度區域的對比
- 這有助於減少光照的影響

In [ ]:
# 練習 2 解答

def gamma_correction(image, gamma=0.5):
    """
    對圖像進行 Gamma Correction
    
    Parameters
    ----------
    image : np.ndarray
        輸入圖像，值範圍 [0, 255]
    gamma : float
        Gamma 值，< 1 會增強暗部，> 1 會增強亮部
    
    Returns
    -------
    np.ndarray
        Gamma 校正後的圖像
    """
    # 正規化到 [0, 1]
    normalized = image / 255.0
    
    # 應用 gamma correction
    corrected = np.power(normalized, gamma)
    
    # 轉回 [0, 255]
    return corrected * 255


def compute_hog_with_gamma(image, gamma=0.5, **kwargs):
    """
    帶 Gamma correction 的 HOG 計算
    """
    # Step 0: Gamma correction
    corrected = gamma_correction(image, gamma)
    
    # 計算 HOG
    return compute_hog(corrected, **kwargs)


# 測試 Gamma correction 的效果
print("=== 測試 Gamma Correction ===")

# 建立有光照變化的測試圖像
person_img = test_images['person']
dark_img = person_img * 0.3  # 很暗的版本

# 比較有無 Gamma correction
features_normal, _ = compute_hog(person_img)
features_dark_no_gamma, _ = compute_hog(dark_img)
features_dark_with_gamma, _ = compute_hog_with_gamma(dark_img, gamma=0.5)

print(f"Normal vs Dark (無 Gamma): {cosine_similarity(features_normal, features_dark_no_gamma):.4f}")
print(f"Normal vs Dark (有 Gamma): {cosine_similarity(features_normal, features_dark_with_gamma):.4f}")

# 視覺化
fig, axes = plt.subplots(2, 3, figsize=(12, 8))

# 原圖
axes[0, 0].imshow(person_img, cmap='gray', vmin=0, vmax=255)
axes[0, 0].set_title('Original')
axes[0, 0].axis('off')

# 暗版本
axes[0, 1].imshow(dark_img, cmap='gray', vmin=0, vmax=255)
axes[0, 1].set_title('Dark (×0.3)')
axes[0, 1].axis('off')

# Gamma 校正後
dark_gamma = gamma_correction(dark_img, gamma=0.5)
axes[0, 2].imshow(dark_gamma, cmap='gray', vmin=0, vmax=255)
axes[0, 2].set_title('Dark + Gamma (γ=0.5)')
axes[0, 2].axis('off')

# Histogram 比較
axes[1, 0].hist(person_img.flatten(), bins=50, alpha=0.7)
axes[1, 0].set_title('Original Histogram')

axes[1, 1].hist(dark_img.flatten(), bins=50, alpha=0.7)
axes[1, 1].set_title('Dark Histogram')

axes[1, 2].hist(dark_gamma.flatten(), bins=50, alpha=0.7)
axes[1, 2].set_title('Dark + Gamma Histogram')

plt.suptitle('Gamma Correction 效果', fontsize=14)
plt.tight_layout()
plt.show()

print("\n觀察：Gamma correction 展開了暗部圖像的直方圖，使其更接近正常曝光")

### 練習 3：HOG 特徵相似度計算

**任務**：實作一個函數來比較兩個圖像的 HOG 特徵相似度

這是物件偵測的基礎——比較待測窗口與模板的相似度。

**提示**：
- 可以用 Cosine Similarity
- 也可以用 Euclidean Distance（較小表示較相似）
- 或 Chi-square distance（對直方圖更適合）

In [ ]:
# 練習 3 解答

def hog_similarity(features1, features2, method='cosine'):
    """
    計算兩個 HOG 特徵向量的相似度
    
    Parameters
    ----------
    features1, features2 : np.ndarray
        HOG 特徵向量
    method : str
        相似度計算方法：'cosine', 'euclidean', 'chi_square'
    
    Returns
    -------
    float
        相似度分數（cosine: 越大越相似，euclidean/chi_square: 越小越相似）
    """
    if method == 'cosine':
        # Cosine similarity: dot(a, b) / (|a| * |b|)
        # 範圍 [-1, 1]，1 表示完全相同
        norm1 = np.linalg.norm(features1)
        norm2 = np.linalg.norm(features2)
        if norm1 == 0 or norm2 == 0:
            return 0
        return np.dot(features1, features2) / (norm1 * norm2)
    
    elif method == 'euclidean':
        # Euclidean distance: ||a - b||_2
        # 0 表示完全相同
        return np.linalg.norm(features1 - features2)
    
    elif method == 'chi_square':
        # Chi-square distance: Σ (a - b)² / (a + b + eps)
        # 對直方圖類型的特徵效果較好
        eps = 1e-10
        diff_sq = (features1 - features2) ** 2
        sum_feat = features1 + features2 + eps
        return np.sum(diff_sq / sum_feat)
    
    else:
        raise ValueError(f"Unknown method: {method}")


# 測試相似度計算
print("=== 測試 HOG 相似度計算 ===")

# 建立測試圖像
person_img = test_images['person']
vertical_img = test_images['vertical']

# 建立變形版本
person_noisy = person_img + np.random.normal(0, 10, person_img.shape)
person_noisy = np.clip(person_noisy, 0, 255)

# 計算 HOG 特徵
features_person, _ = compute_hog(person_img)
features_person_noisy, _ = compute_hog(person_noisy)

# 調整垂直邊緣圖像大小以匹配
vertical_resized = np.zeros_like(person_img)
vertical_resized[:64, :] = vertical_img
vertical_resized[64:, :] = vertical_img
features_vertical, _ = compute_hog(vertical_resized)

# 比較相似度
print("\n各種方法的相似度比較：")
print(f"{'比較對象':<30} {'Cosine':<12} {'Euclidean':<12} {'Chi-square':<12}")
print("-" * 66)

comparisons = [
    ('Person vs Person', features_person, features_person),
    ('Person vs Person+Noise', features_person, features_person_noisy),
    ('Person vs Vertical', features_person, features_vertical),
]

for name, f1, f2 in comparisons:
    cos = hog_similarity(f1, f2, 'cosine')
    euc = hog_similarity(f1, f2, 'euclidean')
    chi = hog_similarity(f1, f2, 'chi_square')
    print(f"{name:<30} {cos:<12.4f} {euc:<12.4f} {chi:<12.4f}")

print("\n解讀：")
print("- Cosine: 越接近 1 越相似")
print("- Euclidean/Chi-square: 越接近 0 越相似")

---

## 整合練習：完整的 HOG 特徵描述子類別

將所有功能整合成一個易用的類別。

In [ ]:
class HOGDescriptor:
    """
    HOG (Histograms of Oriented Gradients) 特徵描述子
    
    實作 Dalal & Triggs (CVPR 2005) 的 HOG 特徵
    
    Attributes
    ----------
    cell_size : int
        Cell 大小（pixels），預設 8
    block_size : int
        Block 大小（cells），預設 2
    num_bins : int
        方向直方圖的 bin 數量，預設 9
    use_gamma : bool
        是否使用 Gamma correction 預處理
    gamma : float
        Gamma 值（僅在 use_gamma=True 時使用）
    normalization : str
        正規化方法：'L2', 'L2-Hys', 或 'none'
    
    Methods
    -------
    compute(image)
        計算圖像的 HOG 特徵
    visualize(image)
        視覺化 HOG 特徵
    similarity(features1, features2, method='cosine')
        計算兩個 HOG 特徵的相似度
    """
    
    def __init__(self, cell_size=8, block_size=2, num_bins=9,
                 use_gamma=False, gamma=0.5, normalization='L2-Hys'):
        """
        初始化 HOG 描述子
        """
        self.cell_size = cell_size
        self.block_size = block_size
        self.num_bins = num_bins
        self.use_gamma = use_gamma
        self.gamma = gamma
        self.normalization = normalization
        
        # 每個 block 的特徵維度
        self.block_dim = block_size * block_size * num_bins
    
    def _gamma_correction(self, image):
        """Gamma correction 預處理"""
        normalized = image / 255.0
        corrected = np.power(normalized + 1e-10, self.gamma)
        return corrected * 255
    
    def _compute_gradients(self, image):
        """計算梯度"""
        H, W = image.shape
        img = image.astype(np.float64)
        
        # Gx
        gx = np.zeros((H, W), dtype=np.float64)
        gx[:, 1:-1] = img[:, 2:] - img[:, :-2]
        gx[:, 0] = img[:, 1] - img[:, 0]
        gx[:, -1] = img[:, -1] - img[:, -2]
        
        # Gy
        gy = np.zeros((H, W), dtype=np.float64)
        gy[1:-1, :] = img[2:, :] - img[:-2, :]
        gy[0, :] = img[1, :] - img[0, :]
        gy[-1, :] = img[-1, :] - img[-2, :]
        
        magnitude = np.sqrt(gx**2 + gy**2)
        direction = np.degrees(np.arctan2(gy, gx)) % 180
        
        return magnitude, direction
    
    def _compute_cell_histogram(self, magnitude, direction):
        """計算 cell 直方圖（soft binning）"""
        bin_width = 180.0 / self.num_bins
        bin_pos = direction / bin_width
        
        bin_left = np.floor(bin_pos).astype(int) % self.num_bins
        bin_right = (bin_left + 1) % self.num_bins
        weight_right = bin_pos - np.floor(bin_pos)
        weight_left = 1 - weight_right
        
        hist = np.zeros(self.num_bins)
        np.add.at(hist, bin_left.flatten(), (magnitude * weight_left).flatten())
        np.add.at(hist, bin_right.flatten(), (magnitude * weight_right).flatten())
        
        return hist
    
    def _normalize_block(self, block, eps=1e-5):
        """Block 正規化"""
        if self.normalization == 'none':
            return block
        
        norm = np.sqrt(np.sum(block**2) + eps)
        normalized = block / norm
        
        if self.normalization == 'L2-Hys':
            normalized = np.minimum(normalized, 0.2)
            norm2 = np.sqrt(np.sum(normalized**2) + eps)
            normalized = normalized / norm2
        
        return normalized
    
    def compute(self, image):
        """
        計算圖像的 HOG 特徵
        
        Parameters
        ----------
        image : np.ndarray
            灰階圖像，shape (H, W)
        
        Returns
        -------
        features : np.ndarray
            HOG 特徵向量
        cell_hists : np.ndarray
            Cell 直方圖，shape (num_cells_y, num_cells_x, num_bins)
        """
        # 預處理
        if self.use_gamma:
            image = self._gamma_correction(image)
        
        H, W = image.shape
        num_cells_y = H // self.cell_size
        num_cells_x = W // self.cell_size
        
        # 計算梯度
        magnitude, direction = self._compute_gradients(image)
        
        # 計算 cell 直方圖
        cell_hists = np.zeros((num_cells_y, num_cells_x, self.num_bins))
        
        for cy in range(num_cells_y):
            for cx in range(num_cells_x):
                y_start = cy * self.cell_size
                x_start = cx * self.cell_size
                
                cell_mag = magnitude[y_start:y_start+self.cell_size,
                                    x_start:x_start+self.cell_size]
                cell_dir = direction[y_start:y_start+self.cell_size,
                                    x_start:x_start+self.cell_size]
                
                cell_hists[cy, cx] = self._compute_cell_histogram(cell_mag, cell_dir)
        
        # Block normalization
        num_blocks_y = num_cells_y - self.block_size + 1
        num_blocks_x = num_cells_x - self.block_size + 1
        
        features = []
        
        for by in range(num_blocks_y):
            for bx in range(num_blocks_x):
                block = cell_hists[by:by+self.block_size,
                                  bx:bx+self.block_size].flatten()
                features.append(self._normalize_block(block))
        
        return np.concatenate(features), cell_hists
    
    def visualize(self, image, ax=None):
        """
        視覺化 HOG 特徵
        
        Parameters
        ----------
        image : np.ndarray
            灰階圖像
        ax : matplotlib.axes.Axes, optional
            要繪製的 axes
        
        Returns
        -------
        fig, ax : matplotlib Figure and Axes
        """
        features, cell_hists = self.compute(image)
        
        if ax is None:
            fig, ax = plt.subplots(figsize=(10, 12))
        else:
            fig = ax.figure
        
        ax.imshow(image, cmap='gray')
        
        num_cells_y, num_cells_x = cell_hists.shape[:2]
        bin_width = 180.0 / self.num_bins
        angles_rad = np.radians(np.arange(self.num_bins) * bin_width + bin_width / 2)
        
        max_val = cell_hists.max() + 1e-5
        
        for cy in range(num_cells_y):
            for cx in range(num_cells_x):
                center_y = cy * self.cell_size + self.cell_size / 2
                center_x = cx * self.cell_size + self.cell_size / 2
                
                hist = cell_hists[cy, cx]
                
                for b in range(self.num_bins):
                    if hist[b] < max_val * 0.05:
                        continue
                    
                    length = (hist[b] / max_val) * self.cell_size / 2
                    angle = angles_rad[b]
                    dx = length * np.cos(angle)
                    dy = length * np.sin(angle)
                    
                    ax.plot([center_x - dx, center_x + dx],
                           [center_y - dy, center_y + dy],
                           'g-', linewidth=1, alpha=0.7)
        
        ax.set_title(f'HOG Visualization\nFeature dim: {len(features)}')
        ax.axis('off')
        
        return fig, ax
    
    @staticmethod
    def similarity(features1, features2, method='cosine'):
        """
        計算兩個 HOG 特徵的相似度
        
        Parameters
        ----------
        features1, features2 : np.ndarray
            HOG 特徵向量
        method : str
            'cosine', 'euclidean', 或 'chi_square'
        
        Returns
        -------
        float
            相似度分數
        """
        return hog_similarity(features1, features2, method)
    
    def get_feature_dim(self, image_height, image_width):
        """
        計算給定圖像大小的特徵維度
        
        Parameters
        ----------
        image_height, image_width : int
            圖像大小
        
        Returns
        -------
        int
            特徵維度
        """
        num_cells_y = image_height // self.cell_size
        num_cells_x = image_width // self.cell_size
        num_blocks_y = num_cells_y - self.block_size + 1
        num_blocks_x = num_cells_x - self.block_size + 1
        return num_blocks_y * num_blocks_x * self.block_dim


# 測試完整類別
print("=== 測試 HOGDescriptor 類別 ===")

# 建立描述子
hog = HOGDescriptor(cell_size=8, block_size=2, num_bins=9,
                    use_gamma=True, normalization='L2-Hys')

print(f"HOG 參數:")
print(f"  Cell size: {hog.cell_size}×{hog.cell_size}")
print(f"  Block size: {hog.block_size}×{hog.block_size} cells")
print(f"  Num bins: {hog.num_bins}")
print(f"  Use gamma: {hog.use_gamma}")
print(f"  Normalization: {hog.normalization}")

# 計算特徵
person_img = test_images['person']
print(f"\n圖像大小: {person_img.shape}")
print(f"預期特徵維度: {hog.get_feature_dim(*person_img.shape)}")

features, cell_hists = hog.compute(person_img)
print(f"實際特徵維度: {len(features)}")

# 視覺化
fig, ax = hog.visualize(person_img)
plt.show()

print("\nHOGDescriptor 類別測試完成！")

---

## 總結

### 本 Notebook 涵蓋的內容

1. **HOG 架構**：
   - Cell → Block → Normalization 的層次結構
   - 標準參數設定（8×8 cell, 2×2 block, 9 bins）

2. **梯度計算**：
   - 簡單的 centered difference
   - Unsigned gradient（0°-180°）

3. **Cell Histogram**：
   - Hard binning vs Soft binning（雙線性內插）
   - 梯度大小加權投票

4. **Block Normalization**：
   - L2 normalization
   - L2-Hys（帶 clipping 的 L2）
   - 處理光照變化的能力

5. **完整實作**：
   - `compute_hog()` 函數
   - `HOGDescriptor` 類別
   - 視覺化和相似度計算

### HOG 的應用場景

- **行人偵測**：HOG + Linear SVM（我們將在 Module 4 實作）
- **物件偵測**：作為滑動窗口 + 分類器的特徵
- **影像分類**：與 Bag of Visual Words 結合

### 下一步

在 **Module 4** 中，我們將：
1. 實作 Linear SVM 分類器
2. 使用 HOG + SVM 進行行人偵測
3. 理解滑動窗口和非極大值抑制

### 關鍵要點

1. **HOG 捕捉形狀資訊**：通過統計局部梯度方向的分布
2. **Block normalization 是關鍵**：處理光照變化
3. **Dense vs Sparse**：HOG 是 dense feature（整張圖），SIFT 是 sparse feature（關鍵點）
4. **HOG 不具旋轉不變性**：但對行人偵測來說，人通常是直立的，所以這不是問題